In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce
from operator import itemgetter
from collections import defaultdict, Counter
import itertools
import math
import unittest
from copy import deepcopy

In [ ]:
# dodac produkcje zwracaja nowe pod grafy <check> 
# dodac do node'ow typu I id parenta, pierwszy node Typu I bedzie zawierac parenta -1 <check>
# layer ma byc lista <check>
# wizualizacja warstwy (brutem) <check>
# dodac check czy mozna wywolac <check>
# czyscic kernel i out przed pushem

In [ ]:
class Id_creator:
    def __init__(self):
        self.last_id = -1
    def get_id(self):
        self.last_id +=1
        return self.last_id
    def __call__(self):
        return self.get_id()
    
    
class CannotExecuteProduction(Exception):
        pass
    
    
class Graph_layers:
    def __init__(self):
        self._node_id_gen =  Id_creator()
        G = nx.Graph()
        G.add_nodes_from([(self._node_id_gen(), {'pos': (0,0),'type': 'e'})])
        self._layers=[[G]]
        
    def get_layer(self, i):
        return self._layers[i]
    
    def add_to_layer(self,i,G):
        return self._layers[i].append(G)
    
    def add_to_last_layer(self,G):
        return self._layers[-1].append(G)
        
    def add_new_layer(self, G_new):
        self._layers.append([G_new])            
        
    def get_last_layer_index(self):
        return len(self._layers) - 1
    
    def get_last_layer(self,i):
        return self._layers[i]
    
    def display_i_layer(self, i):
        G_layer = self._layers[i]
        G = self._layers[i][0] if  len(self._layers[i])==1 else reduce(nx.algorithms.operators.binary.compose,G_layer)
        
        pos = nx.get_node_attributes(G, 'pos')
        nx.draw_networkx(G, pos)
        
    def get_node_id_gen(self):
        return self._node_id_gen

In [ ]:
def conc_duplicates(G):
    org_nodes = list(G.nodes(data=True))

    org_nodes_dict = {}
    for node_id,data in org_nodes:
        try:
            org_nodes_dict[data['pos']].append(node_id)
        except KeyError:
            org_nodes_dict[data['pos']]=[node_id]

    nodes_to_replace = {} #keys nodes to be deleted, values nodes to replace them
    for key,ids in org_nodes_dict.items():
        if len(ids) > 1:
            for id_ in ids[1:]:
                nodes_to_replace[id_] = ids[0]
                
    for old_node, new_node in nodes_to_replace.items():
        neighbors = list(G.neighbors(old_node))
        for n in neighbors:
            G.remove_edge(old_node,n)
            G.add_edge(new_node,n)
        G.remove_node(old_node)
    return G

In [ ]:
def avg_pos(pos1, pos2):
    return ((pos1[0] + pos2[0])/2, (pos1[1] + pos2[1])/2)

def rm_edge_if_exists(G,n1id,n2id):
    try:
        G.remove_edge(n1id,n2id)
    except nx.NetworkXError:
        pass

In [ ]:
def get_graph_i_nodes(G):
    return [key for (key, value) in nx.get_node_attributes(G, 'type').items() if value == 'I']

def get_graph_i_nodes_by_ids(G, ids):
    return [key for (key, value) in nx.get_node_attributes(G, 'type').items() if value == 'I' and key in ids]

def get_parent_of_nodes(G, nodes):
    return list(set([value for (key, value) in nx.get_node_attributes(G, 'parent').items() if key in nodes]))

def get_distance(fst, snd):
    return math.sqrt((fst[0]-snd[0])**2 + (fst[1]-snd[1])**2)

In [ ]:
class node_grouping(defaultdict):
    def __init__(self):
        super(node_grouping, self).__init__(list)

    def select(self, *keys):
        return tuple(self[key] for key in keys)

def group_nodes_by_attr(graph, n_ids, attr, transform=None):
    grouping = node_grouping()
    for n_id in n_ids:
        node = graph.nodes[n_id]
        if attr not in node:
            continue
        key = node[attr]
        if transform:
            key = transform(key)
        grouping[key].append(n_id)
    return grouping

def find_common_e_neighbors(graph, id1, *ids):
    intersection = { n_id for n_id in graph.neighbors(id1) if graph.nodes[n_id]['type'].lower() == 'e' }
    for xid in ids:
        intersection.intersection_update({ n_id for n_id in graph.neighbors(xid) if graph.nodes[n_id]['type'].lower() == 'e' })
    return intersection

def find_common_group_e_neighbors(graph, group1, *groups):
    intersection = { neighbor for n_id in group1 for neighbor in graph.neighbors(n_id) if graph.nodes[neighbor]['type'].lower() == 'e' }
    for group in groups:
        intersection.intersection_update({ neighbor for n_id in group for neighbor in graph.neighbors(n_id) if graph.nodes[neighbor]['type'].lower() == 'e' })
        intersection.difference_update(set(group))
    return intersection - set(group1)

def isapprox(a, b):
    eps = 1e-10
    ax, ay = a
    bx, by = b
    return abs(ax - bx) <= eps and abs(ay - by) <= eps

In [ ]:
def p1(G, base_node_id, n_id_gen, side_len=2, max_random_offset = 0):
    assert(G.nodes[base_node_id]['type'].lower() == 'e' )
    assert(len(G.nodes)==1)
    all_new_nodes = []
    all_new_edges = []
    base_pos = G.nodes[base_node_id]['pos']
    x_offset = ((np.random.random()-0.5) * max_random_offset * 2)
    y_offset = ((np.random.random()-0.5) * max_random_offset * 2)
    i_node_x, i_node_y = base_pos[0], base_pos[1]
    i_node = (n_id_gen(), {'pos': (i_node_x+x_offset, i_node_y-y_offset), 'type': 'I','parent': -1})
    all_new_nodes.append(i_node)
    
    half_side_len = side_len/2
    e_nodes = [
        (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y + half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y + half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y - half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y - half_side_len),'type': 'e'})
    ]
    for i in range(len(e_nodes)):
        all_new_edges.extend([(e_nodes[i][0],e_nodes[i+1 if i + 1 < len(e_nodes) else 0][0])])
        all_new_edges.extend([(i_node[0],e_nodes[i][0])])
    
    all_new_nodes.extend(e_nodes)
    nG = nx.Graph()
    nG.add_nodes_from(all_new_nodes)
    nG.add_edges_from(all_new_edges)
    return nG, i_node

In [ ]:
#     e0 - - - e1
#      | \    / |
#      |  I0    |
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0 - - - n0 - - - e1
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    n1 - - - n2 - - - n3
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2 - - - n4 - - - e3

def p2(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    if len(es) != 4:
        raise CannotExecuteProduction
    e0, e1, e2, e3 = None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
    if not(G.has_edge(e0[0],e1[0]) and\
            G.has_edge(e1[0],e3[0]) and\
            G.has_edge(e3[0],e2[0]) and\
            G.has_edge(e0[0],e2[0]) and\
            G.has_edge(e0[0],base_node_id) and\
            G.has_edge(e1[0],base_node_id) and\
            G.has_edge(e2[0],base_node_id) and\
            G.has_edge(e3[0],base_node_id)\
            ):
        raise CannotExecuteProduction
    #prepare all new verticies accord to map above
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    n0 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type':'e'})
    n1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type':'e'})
    n2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    n3 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type':'e'})
    n4 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos' : avg_pos(e3[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    # add all new edges
    new_edges = [
        (e0[0],n0[0]),(n0[0],e1[0]),
        (e0[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],n1[0]),(n0[0],n2[0]),(e1[0],n3[0]),
        (n1[0],I1[0]),(n2[0],I1[0]), (n2[0],I2[0]),(n3[0],I2[0]),
        (n1[0],n2[0]),(n2[0],n3[0]),
        (n1[0],I3[0]),(n2[0],I3[0]), (n2[0],I4[0]),(n3[0],I4[0]),
        (n1[0],e2[0]),(n2[0],n4[0]),(n3[0],e3[0]),
        (e2[0],I3[0]),(n4[0],I3[0]), (n4[0],I4[0]),(e3[0],I4[0]),
        (e2[0],n4[0]),(n4[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,n0,n1,n2,n3,n4,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG

In [ ]:
#
#      I1  - e1    e1 - I3
#        \\  /        \\ /
#         e2          e2
#        / \\         / \\
#     I2  - e3     e3 - I4
#
#              |
#             \\/
#
#      I1  - e1 - I3
#        \\   |   /
#            e2
#        /   |   \\
#     I2  - e3 - I4
def p7(parent_layer, child_layer, base_node_ids, n_id_gen):
    #finding graph in child_layer
    first, second = None, None
    for graph in child_layer:
        if any(item in graph.nodes for item in base_node_ids):
            if second != None:
                raise CannotExecuteProduction
            if first is None:
                first = graph
            else:
                second = graph

    first_i_nodes = get_graph_i_nodes_by_ids(first, base_node_ids)
    second_i_nodes = get_graph_i_nodes_by_ids(second, base_node_ids)

    #checking each I node has a parent
    if not all('parent' in first.nodes[n] for n in first_i_nodes) or not all('parent' in second.nodes[n] for n in second_i_nodes):
        raise CannotExecuteProduction

    #finding graph in parent_layer
    parents = get_parent_of_nodes(first, first_i_nodes) + get_parent_of_nodes(second, second_i_nodes)
    if len(parents) != 2:
        raise CannotExecuteProduction
    parent_graphs = [graph for graph in parent_layer if all(parent in graph.nodes for parent in parents)]
    if len(parent_graphs) > 1:
        raise CannotExecuteProduction
    parent_graph = parent_graphs[0]
    if not any(fst for fst in parent_graph.neighbors(parents[0]) for snd in parent_graph.neighbors(parents[1]) if fst == snd):
        raise CannotExecuteProduction

    #finding nodes to reduce
    nodes_to_reduce_all = [(x_n, y_n) 
         for x in first_i_nodes 
         for y in second_i_nodes 
         for x_n in first.neighbors(x) 
         for y_n in second.neighbors(y)
         if 0.1 > get_distance(nx.get_node_attributes(first, 'pos')[x_n], nx.get_node_attributes(second, 'pos')[y_n])]
    nodes_to_reduce = list(set(nodes_to_reduce_all))
    
    if len(nodes_to_reduce_all) != 6 or len(nodes_to_reduce) != 3:
        raise CannotExecuteProduction
    
    possible_edges = [True for (x, _) in nodes_to_reduce for (y, _) in nodes_to_reduce if x != y and first.has_edge(x, y)]
    if len(possible_edges) != 4:
        raise CannotExecuteProduction
    possible_edges = [True for (_, x) in nodes_to_reduce for (_, y) in nodes_to_reduce if x != y and second.has_edge(x, y)]
    if len(possible_edges) != 4:
        raise CannotExecuteProduction
        
    
    #creating new correct graph
    reversed_nodes_to_reduce = [(value, key) for (key, value) in nodes_to_reduce]
    mapping = dict(nodes_to_reduce + reversed_nodes_to_reduce)

    new_old_graph = deepcopy(first)
    second_old_graph = nx.relabel_nodes(second, mapping, copy=True)

    nodes_to_add = [(node, values)
                             for (node, values) in second_old_graph.nodes().items() 
                             if node not in mapping.keys()]

    new_old_graph.add_nodes_from(nodes_to_add)
    new_old_graph.add_edges_from(second_old_graph.edges)

    i_nodes = get_graph_i_nodes(new_old_graph)

    return i_nodes, new_old_graph, [first, second]

def apply_P7(parent_layer, child_layer, base_node_ids, n_id_gen):
    i_nodes, new_graph, old_graphs = p7(parent_layer, child_layer, base_node_ids, n_id_gen)
    for graph in old_graphs:
        child_layer.remove(graph)
    child_layer.append(new_graph)

In [ ]:
#    P8
#
#
#     I1   - e1 - I3
#       \  /   \ /
#       e2      e2
#      / \      / \
#   I2  - e3  e4   - I4
#
#          |
#         /
#
#      I1  - e1 - I3
#        \   |   /
#            e2
#        /   |   \
#     I2  - e3 - I4

def p8(top_layer, lower_layer, base_node_ids, n_id_gen):
    assert len(base_node_ids) == 4

    # step 1) select lower graph 
    lower_graphs = { G for n_id in base_node_ids for G in lower_layer if G.has_node(n_id) }
    assert len(lower_graphs) == 1
    lower_graph, = lower_graphs

    assert all(n_id in lower_graph.nodes for n_id in base_node_ids)
    
    # step 2) select lower i nodes and their parents
    lower_is, = group_nodes_by_attr(lower_graph, base_node_ids, 'type', str.lower).select('i')

    grouping = group_nodes_by_attr(lower_graph, lower_is, 'parent')
    assert len(grouping) == 2
    
    (top_i1, lower_i1s), (top_i2, lower_i2s) = grouping.items()
    assert len(lower_i1s) == 2
    assert len(lower_i2s) == 2
    
    # step 3) select top graph
    top_graphs = { G for n_id in (top_i1, top_i2) for G in top_layer if G.has_node(n_id) }
    assert len(top_graphs) == 1
    top_graph, = top_graphs
    
    # step 4) validate top_layer connections
    assert len(find_common_e_neighbors(top_graph, top_i1, top_i2)) >= 1
    
    # step 5) find lower e nodes candidates
    e1_candidates = find_common_e_neighbors(lower_graph, *lower_i1s)
    e2_candidates = find_common_e_neighbors(lower_graph, *lower_i2s)

    e0_candidates = find_common_group_e_neighbors(lower_graph, lower_i1s, lower_i2s, e1_candidates, e2_candidates)

    e3_candidates = find_common_group_e_neighbors(lower_graph, lower_i1s, e1_candidates) - e0_candidates
    e4_candidates = find_common_group_e_neighbors(lower_graph, lower_i2s, e2_candidates) - e0_candidates

    # step 6) solve constraints on es
    pos = lambda n_id: lower_graph.nodes[n_id]['pos']

    solutions = [
        (e0, e1, e2, e3, e4) 
        for e0 in e0_candidates 
        for e1 in e1_candidates 
        for e2 in e2_candidates 
        for e3 in e3_candidates 
        for e4 in e4_candidates
        if isapprox(pos(e1), pos(e2))
        and isapprox(pos(e3), pos(e4))
        and isapprox(pos(e1), avg_pos(pos(e0), pos(e3)))
    ]
    assert len(solutions) == 1, "can not apply production or production is ambiguous"
    (e0, e1, e2, e3, e4), = solutions

    # step 7) create graph (merge e1, e2 and e3, e4)
    new_graph = nx.contracted_nodes(lower_graph, e1, e2)
    new_graph = nx.contracted_nodes(new_graph, e3, e4)
    return new_graph, lower_graph

def apply_P8(parent_layer, child_layer, base_node_ids, n_id_gen):
    new_graph, old_graph = p8(parent_layer, child_layer, base_node_ids, n_id_gen)
    child_layer.remove(old_graph)
    child_layer.append(new_graph)

In [ ]:
graph_layers = Graph_layers()
graph_layers.display_i_layer(0)

In [ ]:
base_node = list(graph_layers.get_layer(0)[0].nodes(data=True))[0]
#indexing from 0 
first_layer_G = graph_layers.get_layer(0)[0].copy()

#apply p1 production
G, i_node = p1(first_layer_G, base_node[0], graph_layers.get_node_id_gen()) 
graph_layers.add_new_layer(G)
graph_layers.display_i_layer(1)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(1)[0]
nG = p2(G, 1, graph_layers.get_node_id_gen())
graph_layers.add_new_layer(nG)
graph_layers.display_i_layer(2)

In [ ]:
#apply p2 production 4 times
G = graph_layers.get_layer(2)[0]
graph_layers.add_new_layer( p2(G, 15, graph_layers.get_node_id_gen()))
graph_layers.add_to_last_layer(p2(G, 16, graph_layers.get_node_id_gen()))
graph_layers.add_to_last_layer(p2(G, 17, graph_layers.get_node_id_gen()))
graph_layers.add_to_last_layer(p2(G, 18, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
apply_P7(graph_layers.get_layer(2), graph_layers.get_layer(3), [29,31,41,43], graph_layers.get_node_id_gen())
apply_P7(graph_layers.get_layer(2), graph_layers.get_layer(3), [55,57,67,69], graph_layers.get_node_id_gen())
graph_layers.display_i_layer(3)

In [ ]:
apply_P7(graph_layers.get_layer(2), graph_layers.get_layer(3), [43,44,67,68], graph_layers.get_node_id_gen())
graph_layers.display_i_layer(3)

In [ ]:
apply_P8(graph_layers.get_layer(2), graph_layers.get_layer(3), [30,31,54,55], graph_layers.get_node_id_gen())
graph_layers.display_i_layer(3)

In [ ]:
class P8Tests(unittest.TestCase):
    def valid_input_components(self):
        top_nodes = [
            (1,{'pos': (0,0),'type' : 'e'}),
            (2,{'pos': (-1,0),'type' : 'i'}),
            (3,{'pos': (1,0),'type' : 'i'})
        ]
        top_edges = [(1, 2), (1, 3)]
        lower_nodes = [
            (20, {'pos': (0, 1), 'type': 'e'}),
            (21, {'pos': (0, 0), 'type': 'e'}),
            (23, {'pos': (0, 0), 'type': 'e'}),
            (22, {'pos': (0,-1), 'type': 'e'}),
            (24, {'pos': (0,-1), 'type': 'e'}),

            (10, {'pos': (-0.5,0.5), 'type': 'i', 'parent': 2}),
            (12, {'pos': ( 0.5,0.5), 'type': 'i', 'parent': 3}),

            (11, {'pos': (-0.5,-0.5), 'type': 'i', 'parent': 2}),
            (13, {'pos': ( 0.5,-0.5), 'type': 'i', 'parent': 3}),
        ]
        lower_edges = [
            (20,21), (20,23), 
            (21,22), (23,24),
            (10,20), (10,21),
            (12,20), (12,23),
            (11,21), (11,22),
            (13,23), (13,24),
        ]
        return top_nodes, top_edges, lower_nodes, lower_edges
        
    def valid_output_components(self):
        top_nodes = [
            (1,{'pos': (0,0),'type' : 'e'}),
            (2,{'pos': (-1,0),'type' : 'i'}),
            (3,{'pos': (1,0),'type' : 'i'})
        ]
        top_edges = [(1, 2), (1, 3)]
        lower_nodes = [
            (20, {'pos': (0, 1), 'type': 'e'}),
            (21, {'pos': (0, 0), 'type': 'e'}),
            (22, {'pos': (0,-1), 'type': 'e'}),

            (10, {'pos': (-0.5,0.5), 'type': 'i', 'parent': 2}),
            (12, {'pos': ( 0.5,0.5), 'type': 'i', 'parent': 3}),

            (11, {'pos': (-0.5,-0.5), 'type': 'i', 'parent': 2}),
            (13, {'pos': ( 0.5,-0.5), 'type': 'i', 'parent': 3}),
        ]
        lower_edges = [
            (20,21), (20,21), 
            (21,22),
            (10,20), (10,21),
            (12,20), (12,21),
            (11,21), (11,22),
            (13,21), (13,22),
        ]
        return top_nodes, top_edges, lower_nodes, lower_edges   
        
    def create_graph(self, top_nodes, top_edges, lower_nodes, lower_edges):
        graph = Graph_layers()
        top_graph = nx.Graph()
        top_graph.add_nodes_from(top_nodes)
        top_graph.add_edges_from(top_edges)
        graph.add_new_layer(top_graph)
        
        lower_graph = nx.Graph()
        lower_graph.add_nodes_from(lower_nodes)
        lower_graph.add_edges_from(lower_edges)
        graph.add_new_layer(lower_graph)

        return graph
    
    def test_applying_production_to_minimum_valid_graph(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        graph = self.create_graph(top_nodes, top_edges, lower_nodes, lower_edges)
    
        apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())
    
        valid_output = self.create_graph(*self.valid_output_components())
        self.assertTrue(nx.is_isomorphic(graph.get_layer(2)[0], valid_output.get_layer(2)[0], node_match=lambda n1,n2: n1['type'] == n2['type'] and n1.get('parent', -2) == n2.get('parent', -2)))
    
    def test_should_fail_without_node(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        for node in lower_nodes:
            lower_nodes_missing = list(lower_nodes)
            lower_nodes_missing.remove(node)
            n_id, _ = node
            lower_edges_missing = [edge for edge in lower_edges if n_id not in edge]

            graph = self.create_graph(top_nodes, top_edges, lower_nodes_missing, lower_edges_missing)
            with self.assertRaises(AssertionError):
                apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())
    
    def test_should_fail_without_edge(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        for edge in lower_edges:
            lower_edges_missing = list(lower_edges)
            lower_edges_missing.remove(edge)
        
            graph = self.create_graph(top_nodes, top_edges, lower_nodes, lower_edges_missing)
            with self.assertRaises(AssertionError):
                apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())         
    
    def test_should_fail_with_invalid_label(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        
        for i, (n_id, meta) in enumerate(lower_nodes):
            lower_nodes_invalid = deepcopy(lower_nodes)
            del lower_nodes_invalid[i]
            meta['type'] = 'i' if meta['type'] == 'e' else 'e'
            lower_nodes_invalid.append((n_id, meta))

            graph = self.create_graph(top_nodes, top_edges, lower_nodes_invalid, lower_edges)
            with self.assertRaises(AssertionError):
                apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())   
        
    def test_should_fail_with_invalid_coords(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        lower_nodes.remove((20, {'pos': (0, 1), 'type': 'e'}))
        lower_nodes.append((20, {'pos': (0, 100), 'type': 'e'}))
        
        graph = self.create_graph(top_nodes, top_edges, lower_nodes, lower_edges)
        with self.assertRaises(AssertionError):
            apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())  
    
    def test_should_work_as_subgraph(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        lower_nodes += [
            (14, {'pos': (-0.5,0.5), 'type': 'i', 'parent': 2}),
            (15, {'pos': (-0.5,0.5), 'type': 'i', 'parent': 3}),
        ]
        lower_edges += [(20, 15), (20, 14), (15, 13), (24, 15)]
        graph = self.create_graph(top_nodes, top_edges, lower_nodes, lower_edges)
        apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())

In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
apply_P7(graph_layers.get_layer(2), graph_layers.get_layer(3), [29,31,41,43], graph_layers.get_node_id_gen())
apply_P7(graph_layers.get_layer(2), graph_layers.get_layer(3), [55,57,67,69], graph_layers.get_node_id_gen())
graph_layers.display_i_layer(3)

In [ ]:
apply_P7(graph_layers.get_layer(2), graph_layers.get_layer(3), [43,44,67,68], graph_layers.get_node_id_gen())
graph_layers.display_i_layer(3)

In [ ]:
apply_P8(graph_layers.get_layer(2), graph_layers.get_layer(3), [30,31,54,55], graph_layers.get_node_id_gen())
graph_layers.display_i_layer(3)

In [ ]:
class P8Tests(unittest.TestCase):
    def valid_input_components(self):
        top_nodes = [
            (1,{'pos': (0,0),'type' : 'e'}),
            (2,{'pos': (-1,0),'type' : 'i'}),
            (3,{'pos': (1,0),'type' : 'i'})
        ]
        top_edges = [(1, 2), (1, 3)]
        lower_nodes = [
            (20, {'pos': (0, 1), 'type': 'e'}),
            (21, {'pos': (0, 0), 'type': 'e'}),
            (23, {'pos': (0, 0), 'type': 'e'}),
            (22, {'pos': (0,-1), 'type': 'e'}),
            (24, {'pos': (0,-1), 'type': 'e'}),

            (10, {'pos': (-0.5,0.5), 'type': 'i', 'parent': 2}),
            (12, {'pos': ( 0.5,0.5), 'type': 'i', 'parent': 3}),

            (11, {'pos': (-0.5,-0.5), 'type': 'i', 'parent': 2}),
            (13, {'pos': ( 0.5,-0.5), 'type': 'i', 'parent': 3}),
        ]
        lower_edges = [
            (20,21), (20,23), 
            (21,22), (23,24),
            (10,20), (10,21),
            (12,20), (12,23),
            (11,21), (11,22),
            (13,23), (13,24),
        ]
        return top_nodes, top_edges, lower_nodes, lower_edges
        
    def valid_output_components(self):
        top_nodes = [
            (1,{'pos': (0,0),'type' : 'e'}),
            (2,{'pos': (-1,0),'type' : 'i'}),
            (3,{'pos': (1,0),'type' : 'i'})
        ]
        top_edges = [(1, 2), (1, 3)]
        lower_nodes = [
            (20, {'pos': (0, 1), 'type': 'e'}),
            (21, {'pos': (0, 0), 'type': 'e'}),
            (22, {'pos': (0,-1), 'type': 'e'}),

            (10, {'pos': (-0.5,0.5), 'type': 'i', 'parent': 2}),
            (12, {'pos': ( 0.5,0.5), 'type': 'i', 'parent': 3}),

            (11, {'pos': (-0.5,-0.5), 'type': 'i', 'parent': 2}),
            (13, {'pos': ( 0.5,-0.5), 'type': 'i', 'parent': 3}),
        ]
        lower_edges = [
            (20,21), (20,21), 
            (21,22),
            (10,20), (10,21),
            (12,20), (12,21),
            (11,21), (11,22),
            (13,21), (13,22),
        ]
        return top_nodes, top_edges, lower_nodes, lower_edges   
        
    def create_graph(self, top_nodes, top_edges, lower_nodes, lower_edges):
        graph = Graph_layers()
        top_graph = nx.Graph()
        top_graph.add_nodes_from(top_nodes)
        top_graph.add_edges_from(top_edges)
        graph.add_new_layer(top_graph)
        
        lower_graph = nx.Graph()
        lower_graph.add_nodes_from(lower_nodes)
        lower_graph.add_edges_from(lower_edges)
        graph.add_new_layer(lower_graph)

        return graph
    
    def test_applying_production_to_minimum_valid_graph(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        graph = self.create_graph(top_nodes, top_edges, lower_nodes, lower_edges)
    
        apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())
    
        valid_output = self.create_graph(*self.valid_output_components())
        self.assertTrue(nx.is_isomorphic(graph.get_layer(2)[0], valid_output.get_layer(2)[0], node_match=lambda n1,n2: n1['type'] == n2['type'] and n1.get('parent', -2) == n2.get('parent', -2)))
    
    def test_should_fail_without_node(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        for node in lower_nodes:
            lower_nodes_missing = list(lower_nodes)
            lower_nodes_missing.remove(node)
            n_id, _ = node
            lower_edges_missing = [edge for edge in lower_edges if n_id not in edge]

            graph = self.create_graph(top_nodes, top_edges, lower_nodes_missing, lower_edges_missing)
            with self.assertRaises(AssertionError):
                apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())
    
    def test_should_fail_without_edge(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        for edge in lower_edges:
            lower_edges_missing = list(lower_edges)
            lower_edges_missing.remove(edge)
        
            graph = self.create_graph(top_nodes, top_edges, lower_nodes, lower_edges_missing)
            with self.assertRaises(AssertionError):
                apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())         
    
    def test_should_fail_with_invalid_label(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        
        for i, (n_id, meta) in enumerate(lower_nodes):
            lower_nodes_invalid = deepcopy(lower_nodes)
            del lower_nodes_invalid[i]
            meta['type'] = 'i' if meta['type'] == 'e' else 'e'
            lower_nodes_invalid.append((n_id, meta))

            graph = self.create_graph(top_nodes, top_edges, lower_nodes_invalid, lower_edges)
            with self.assertRaises(AssertionError):
                apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())   
        
    def test_should_fail_with_invalid_coords(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        lower_nodes.remove((20, {'pos': (0, 1), 'type': 'e'}))
        lower_nodes.append((20, {'pos': (0, 100), 'type': 'e'}))
        
        graph = self.create_graph(top_nodes, top_edges, lower_nodes, lower_edges)
        with self.assertRaises(AssertionError):
            apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())  
    
    def test_should_work_as_subgraph(self):
        top_nodes, top_edges, lower_nodes, lower_edges = self.valid_input_components()
        lower_nodes += [
            (14, {'pos': (-0.5,0.5), 'type': 'i', 'parent': 2}),
            (15, {'pos': (-0.5,0.5), 'type': 'i', 'parent': 3}),
        ]
        lower_edges += [(20, 15), (20, 14), (15, 13), (24, 15)]
        graph = self.create_graph(top_nodes, top_edges, lower_nodes, lower_edges)
        apply_P8(graph.get_layer(1), graph.get_layer(2), [10,11,12,13], graph.get_node_id_gen())

In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)